# La méthode de la puissance 

In [ ]:
import numpy as np
import scipy as sp
import numpy.linalg as npl
import scipy.linalg as spl
import scipy.sparse.linalg as sspl
M = np.load("Matrice.npy")

L'objectif de ce travail l'observation du comportement de diverses stratégies 
associées à la méthode de la puissance, ou de la puissance inverse.

On se placera dans le cas ou la matrice $M$ dont on veut calculer une valeur
propre est symétrique.


On va s'intéresser à la méthode de la puissance inverse pour le calcul 
de la plus petite valeur propre d'une matrice $M$  qui vous est fournie, 
dans le fichier Matrice.mat. La matrice $M$ est symétrique 
définie positive. 
Comme nous voulons comparer différentes méthodes, le vecteur initial des 
iterations 
sera un argument des fonctions mettant en oeuvre les différentes méthodes.

#### Les fonctions PinvInf et PinvN2

Créer les fonction PinvInf et PinvN2. La première de ces fonctions mettra en oeuvre l'algorithme de la table 1.1, 
page 13 du polycopié, et la seconde adaptera cette dernière mais pour la norme infinie. 
Pour chaque methode, le test de convergence se fera sur l'écart entre deux itérés successifs de la valeur propre approximée. Le nombre maximum
d'iterations sera limité à deux fois la taille du problème.

In [ ]:
def PinvInf(A,x,tol):
    # Arguments en entrée:
    # A : la matrice
    # x : le vecteur initial
    # tol : le seuil de précision de la méthode
    # Arguments en sortie:
    # l : la plus petite valeur propre de la matrice A
    # u : le vecteur propre associé à l
    # Niter : le nombre d'itérations de la méthode
    
    Niter = 0
    x = x/npl.norm(x,np.inf)
    P,L,U = spl.lu(A)
    l = tol
    ltmp = 0
    while abs(l-ltmp)>tol*abs(l):
        Niter += 1
        ltmp = l
        if Niter>2*np.shape(A)[0]:
            break
        y = spl.solve_triangular(L,x,lower = 1)
        x = spl.solve_triangular(U,y)
        j = np.argmax(np.abs(x))
        l = x[j]
        x = x/abs(l)
    return 1./l,x,Niter

x0 = np.random.rand(np.shape(M)[0])
l,u,Niter = PinvInf(M,x0,1e-10)
print(Niter)
print(l)

In [ ]:
def PinvN2(A,x,tol):
    # Arguments en entrée:
    # A : la matrice
    # x : le vecteur initial
    # tol : le seuil de précision de la méthode
    # Arguments en sortie:
    # l : la plus petite valeur propre de la matrice A
    # u : le vecteur propre associé à l
    # Niter : le nombre d'itérations de la méthode
    
    Niter = 0
    x = x/npl.norm(x)
    P,L,U = spl.lu(A)
    l = tol
    ltmp = 0
    while abs(l-ltmp)>tol*abs(l):
        Niter += 1
        ltmp = l
        if Niter>2*np.shape(A)[0]:
            break
        y = spl.solve_triangular(L,x,lower = 1)
        z = spl.solve_triangular(U,y)
        l = np.dot(x,z)
        x = z/npl.norm(z)
    return 1./l,x,Niter

x0 = np.random.rand(np.shape(M)[0])
l,u,Niter = PinvN2(M,x0,1e-10)
print(Niter)
print(l)

#### Les fonctions TinvInf et TinvN2

Modifier ces fonctions pour qu'elles mettent en oeuvre une stratégie
de translation-inversion de façon à chercher la valeur propre la plus
proche d'un nombre donné. Vous appelerez ces fonctions TinvInf et TinvN2 ; cette stratégie est décrite à la section 1.2.3, page 17 du polycopié. On ajoutera le paramètre $\sigma $ à la liste des variables en entrée.

In [ ]:
def TinvInf(A,x,sigma,tol):
    # Arguments en entrée:
    # A : la matrice
    # x : le vecteur initial
    # sigma : la valeur du shift
    # tol : le seuil de précision de la méthode
    # Arguments en sortie:
    # l : la valeur propre la plus proche de sigma
    # u : le vecteur propre associé à l
    # Niter : le nombre d'itérations de la méthode
    
    Niter = 0
    x = x/npl.norm(x)
    P,L,U = spl.lu(A-sigma*np.eye(np.shape(A)[0]))
    l = tol
    ltmp = 0
    while abs(l-ltmp)>tol*abs(l):
        Niter += 1
        ltmp = l
        if Niter>2*np.shape(A)[0]:
            break
        y = spl.solve_triangular(L,x,lower = 1)
        x = spl.solve_triangular(U,y)
        j = np.argmax(np.abs(x))
        l = x[j]
        x = x/abs(l)
    return 1./l+sigma,x,Niter

x0 = np.random.rand(np.shape(M)[0])
l,u,Niter = TinvInf(M,x0,0.01,1e-10)
print(l)
print(Niter)

In [ ]:
def TinvN2(A,x,sigma,tol):
    # Arguments en entrée:
    # A : la matrice
    # x : le vecteur initial
    # sigma : la valeur du shift
    # tol : le seuil de précision de la méthode
    # Arguments en sortie:
    # l : la valeur propre la plus proche de sigma
    # u : le vecteur propre associé à l
    # Niter : le nombre d'itérations de la méthode
    
    Niter = 0
    x = x/npl.norm(x)
    P,L,U = spl.lu(A-sigma*np.eye(np.shape(A)[0]))
    l = tol
    ltmp = 0
    while abs(l-ltmp)>tol*abs(l):
        Niter += 1
        ltmp = l
        if Niter>2*np.shape(A)[0]:
            break
        y = spl.solve_triangular(L,x,lower = 1)
        z = spl.solve_triangular(U,y)
        l = np.dot(x,z)
        x = z/npl.norm(z)
    return 1./l+sigma,x,Niter


x0 = np.random.rand(np.shape(M)[0])

l,u,Niter = TinvN2(M,x0,0.01,1e-10)
print(l)
print(Niter)

#### Test de la vitesse de convergence des méthodes

On calcule l'ensemble des valeurs propres de la matrice $M$ à l'aide de la fonction eig de numpy.linalg.

In [ ]:
valP,vecP = npl.eig(M)
valP = sorted(valP,reverse = True)

 Tester la vitesse de convergence vers la 4ème valeur propre selon
$\sigma $, lorsqu'elles sont 
  rangées suivant l'ordre décroissant ; on choisira des valeurs de  de 
la forme 
$ \sigma = \left( \frac{\lambda_4 + \lambda _5}{2} \right) 
 + \frac{i}{10}\, \left( \frac{\lambda_4 - \lambda _5}{2} \right) $, 
$1\leq i \leq 9$.

On pourra éventuellement comparer le nombre d'itération nécessaire
à la convergence avec l'estimation que l'on peut en déduire 
et qui est donnée dans la section 3.1.3.

In [ ]:
sigma_arr = (valP[3]+valP[4])/2 + (np.array(range(9))+1)*(valP[3]-valP[4])/20
Niter_Inf_arr,l_Inf_arr = np.zeros(9),np.zeros(9)
Niter_N2_arr,l_N2_arr = np.zeros(9),np.zeros(9)
Vit_conv = np.zeros(9)

x = np.random.rand(272)
for i in range(9):
    Vit_conv[i] = abs((valP[3]-sigma_arr[i])/(valP[4]-sigma_arr[i]))
    l,u,Niter = TinvInf(M,x,sigma_arr[i],1e-10)
    Niter_Inf_arr[i] = Niter
    l_Inf_arr[i] = l
    l,u,Niter = TinvN2(M,x,sigma_arr[i],1e-10)
    Niter_N2_arr[i] = Niter
    l_N2_arr[i] = l
print("Valeurs propres approximées pour la norme infinie:")
print(l_Inf_arr)
print("Valeurs propres approximées pour la norme 2:")
print(l_N2_arr)
print("Nombre total d'itérations pour la norme infinie:")
print(Niter_Inf_arr)
print("Nombre total d'itérations pour la norme 2:")
print(Niter_N2_arr)

#### La fonction Rayleigh

Ecrire une fonction Rayleigh qui modifie TinvN2 
suivant l'algorithme du tableau 1.3, page 19 du polycopié.

In [ ]:
def Rayleigh(A,x,tol):
    # Arguments en entrée:
    # A : la matrice
    # x : le vecteur initial
    # tol : le seuil de précision de la méthode
    # Arguments en sortie:
    # l : la valeur propre associée à u
    # u : le vecteur propre le plus proche de x
    # Niter : le nombre d'itérations de la méthode
    
    Niter = 0
    x = x/npl.norm(x)
    l = np.dot(x,np.dot(A,x))
    ltmp = 0
    while abs(l-ltmp)>tol*abs(ltmp):
        Niter += 1
        ltmp = l
        if Niter>2*np.shape(A)[0]:
            break
        x = npl.solve(A-l*np.eye(np.shape(A)[0]),x)
        x = x/npl.norm(x,2)
        l = np.dot(x,np.dot(A,x))
    return l,x,Niter

x0 = np.random.rand(np.shape(M)[0])
l,u,Niter = Rayleigh(M,x0,1e-10)
print(l)
print(Niter)

Proposez une fonction qui combine PinvN2 et  Rayleigh
pour le calcul de la plus petite valeur propre. Comparez les différentes 
méthodes pour la recherche de la plus petite valeur propre.

In [ ]:
def Combine(A,x,tol_Rayleigh,tol):
    # Arguments en entrée:
    # A : la matrice
    # x : le vecteur initial
    # tol_Rayleigh : le seuil à partir du quel on passe à la méthode de Rayleigh
    # tol : le seuil de précision de la méthode
    # Arguments en sortie:
    # l : la valeur propre associée à u
    # u : le vecteur propre le plus proche de x
    # Niter : le nombre d'itérations de la méthode
    
    Niter = 0
    x = x/npl.norm(x)
    P,L,U = spl.lu(A)
    l = tol
    ltmp = 0
    while abs(l-ltmp)>tol_Rayleigh*abs(l):
        Niter += 1
        ltmp = l
        if Niter>2*np.shape(A)[0]:
            break
        y = spl.solve_triangular(L,x,lower = 1)
        z = spl.solve_triangular(U,y)
        l = np.dot(x,z)
        x = z/npl.norm(z)
    while abs(l-ltmp)>tol*abs(ltmp):
        Niter += 1
        ltmp = l
        if Niter>2*np.shape(A)[0]:
            break
        x = npl.solve(A-l*np.eye(np.shape(A)[0]),x)
        x = x/npl.norm(x,2)
        l = np.dot(x,np.dot(A,x))
    return l,x,Niter

a = np.random.random(272)
l,u,Niter1 = Combine(M,a,1e-3,1e-10)
l,u,Niter2 = PinvN2(M,a,1e-10)
print("Itérations pour la puissance inverse:",Niter2)
print("Itérations pour la méthode combinée:",Niter1)
print("Gaine en itérations:",Niter2-Niter1)

#### Calculer plusieurs valeurs propres

L'objectif est de calculer $d$ valeurs propres d'une même matrice symétrique, pour cela à l'itération $k$ on effectuera un algorithme de puissance itérée sur $d$ vecteurs en même temps : $(x_1^k,x_2^k,..x_d^k)$. On calcule donc les vecteurs $$y_i^k=A x_i^k \text{ pour } i=1..d.$$ Finalement, comme on sait qu'au final les vecteurs propres de $A$ sont orthogonaux, on orthonormalise les vecteurs $(y_1^k,y_2^k,..y_d^k)$ par un procédé standard de Gram-Schmidt et on prend cette base orthonormalisée comme valeur de $(x_1^{k+1},x_2^{k+1},..x_d^{k+1})$.
On vous donne ci-dessous un procédé de Gram-Schmidt qui prend une suite de vecteurs (mise sous forme de tableau à deux dimensions) et qui l'orthonormalise.

In [ ]:
def GramSchmidt(Y) :
    [n,p]=Y.shape
    actual_index=0
    for i in np.arange(p) :
        for j in np.arange(i) :
            Y[:,i]-=np.dot(Y[:,i],Y[:,j])*Y[:,j]
        Y[:,i]/=np.linalg.norm(Y[:,i])

A = np.reshape(np.arange(20,dtype=float),(5,4))
print(A)
GramSchmidt(A)
print(A.T.dot(A))

Implémenter un algorithme de calcul des premières valeurs propres. Tester avec la matrice symétrique du Laplacien avec conditions aux bord de Dirichet donné dans la cellule suivante. La formule des premières valeurs propres du Laplacien à $n$ points est :
$$ l[k]= 4(n+1)^2sin^2\left(\frac{k\pi}{2(n+1)}\right)$$

In [ ]:
def Laplacian(n) :
    return (n+1)**2*sp.sparse.diags([2*np.ones(n),-1*np.ones(n-1),-1*np.ones(n-1)], [0, -1, 1]).tocsc()
def PinvN2(A,X,tol):
    # Arguments en entrée:
    # A : la matrice
    # X : le vecteur initial (possiblement une matrice)
    # tol : le seuil de précision de la méthode
    # Arguments en sortie:
    # l : les plus petites valeurs propres de la matrice A
    # u : les vecteur propres associé à l
    # Niter : le nombre d'itérations de la méthode
    
    Niter = 0
    GramSchmidt(X)
    p = np.shape(X)[1]
    l = tol*np.ones((p,1))
    ltmp = np.zeros((p,1))
    while npl.norm(l-ltmp,np.inf)>tol*npl.norm(ltmp,np.inf):
        Niter += 1
        ltmp = l
        if Niter>2*np.shape(A)[0]:
            break
        X = sspl.spsolve(A,X)
        GramSchmidt(X)
        B = X.T.dot(A.dot(X))
        l = np.diag(B)
    return l,X,Niter

n = 500
p = 5
X = np.random.rand(n,p)
l,X,niter=PinvN2(Laplacian(n),X,1e-6)
lexact = 4*((n+1)*np.sin(np.arange(1,n+1)*np.pi/(2*(n+1))))**2
print("Erreur entre les valeurs propres calculées et les valeurs propres exactes:")
print(l - lexact[0:5])